In [1]:
import numpy as np
import scipy.linalg
import cvxpy as cp
from numpy.linalg import matrix_power

A Robust Learning Approach for Regression Models Based on Distributionally Robust Optimization

In [2]:
def dlqr(A,B,Q,R):
    """
    Calculates the optimal gain matrix K for system
    x[k+1] = A x[k] + B u[k]
    with cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k], such that
    u[k] = -K x[k]
    
    Returns: 
        -K
    """
    # first, solve the ricatti equation
    P = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*P*B+R)*(B.T*P*A))
    return -K

In [3]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [4]:
def RK4_np(f, x, u, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x, u)
    k2 = f(t + h / 2, x + h / 2 * k1, u)
    k3 = f(t + h / 2, x + h / 2 * k2, u)
    k4 = f(t + h, x + h * k3, u)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [5]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

Continuous-time system

$\begin{aligned}\left[\begin{array}{c}\dot{x}_{1}(t) \\ \dot{x}_{2}(t)\end{array}\right] &=\left[\begin{array}{cc}0 & 1 \\ -k_{2} / m & -k_{1} / m\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right]+\left[\begin{array}{c}0 \\ 1 / m\end{array}\right] u(t) \\ y(t) &=\left[\begin{array}{ll}1 & 0\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right] \end{aligned}$

In [6]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

In [7]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [8]:
Ak,Bk = disc_linear_system(A,B,delta_t)

In [9]:
Ak

array([[ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897]])

In [10]:
Bk

array([[0.00431034],
       [0.04310345]])

In [11]:
Q = np.diag([10,1])
Qf = np.diag([15,1])
R = np.diag([1])

In [12]:
neg_K = dlqr(Ak,Bk,Q,R)

In [13]:
neg_K

matrix([[-1.64465128, -1.02359576]])

In [14]:
N_sim = 100

x_init = np.array([[2],[0]])
t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t

x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    uk = neg_K @ xk
    u_list +=  uk.flatten().tolist() 
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1

In [15]:
u_list

[[-3.289302564456352],
 [-2.914942510901101],
 [-2.5598110182383618],
 [-2.2263935494494858],
 [-1.9163535315592184],
 [-1.6306546787177174],
 [-1.369672758698418],
 [-1.1332968450937406],
 [-0.9210202675311228],
 [-0.7320216075616044],
 [-0.5652361925567164],
 [-0.41941861782554035],
 [-0.2931968818221258],
 [-0.1851187540790109],
 [-0.09369101342841346],
 [-0.017412197937609175],
 [0.04520049970282325],
 [0.09558957459547412],
 [0.13514019742854322],
 [0.16516728509624534],
 [0.1869061255229036],
 [0.20150606471958082],
 [0.21002680236539475],
 [0.2134368811374406],
 [0.21261399383289373],
 [0.2083467703985401],
 [0.20133774379984534],
 [0.19220722883086472],
 [0.18149788119473242],
 [0.16967973526549884],
 [0.15715554774320428],
 [0.14426630086558034],
 [0.13129674292442273],
 [0.11848086557844487],
 [0.10600723691806505],
 [0.09402412650932193],
 [0.08264437383307388],
 [0.07194996476618828],
 [0.06199629215829093],
 [0.052816086281720964],
 [0.04442300911719047],
 [0.0368149132265

In [16]:
x_list

[[2, 0],
 [1.9826689238506545, -0.337883851188942],
 [1.9349862198417493, -0.6082152491804027],
 [1.8633130681783237, -0.8187867821451399],
 [1.7732520651954105, -0.9769752784440978],
 [1.669691328807902, -1.089693169266517],
 [1.5568526124067068, -1.163356605030776],
 [1.4383418800523347, -1.2038677983761676],
 [1.3172010373594736, -1.2166092885486843],
 [1.1959597318770925, -1.2064480423099309],
 [1.0766863343887605, -1.1777475217519346],
 [0.9610373890339324, -1.134386056870654],
 [0.8503049763687974, -1.0797800583539778],
 [0.7454615705111942, -1.0169107922407186],
 [0.6472020905436895, -0.9483536118440088],
 [0.5559829486632923, -0.8763087029491551],
 [0.4720579844946872, -0.8026325454778964],
 [0.3955112478724361, -0.7288694285283872],
 [0.3262866525834703, -0.6562824761564078],
 [0.2642145723515755, -0.5858837488486628],
 [0.20903548900109586, -0.5184630808900762],
 [0.16042083245903396, -0.454615397390631],
 [0.11799117417557238, -0.3947663273396302],
 [0.08133195071989681, -0.

### System formulation
#### plant:

$$
\begin{aligned}
x_{k+1} &=A x_{k}+B u_{k} + C w_{k}\\
y_{k} &=D x_{k} + E w_{k}
\end{aligned}
$$
with $x_k \in \mathbb{R}^{n}, u_k \in \mathbb{R}^{m}, y_k \in \mathbb{R}^{r}, w_k \in \mathbb{R}^{d}$. Initial state $x_0$.


#### model:

$$
\begin{aligned}
\hat{x}_{k+1} &=A \hat{x}_{k}+B u_{k}\\
\hat{y}_{k} &=D \hat{x}_{k}
\end{aligned}
$$

Initial state $0$


### System behabior in N step
Define: $$
\boldsymbol{X}=\left[\begin{array}{llll}
x_{k}^{\top}, x_{k+1}^{\top} & \ldots & x_{k+N}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{U}=\left[\begin{array}{llll}
u_{k}^{\top} & u_{k+1}^{\top} & \ldots & u_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}=\left[\begin{array}{llll}
y_{k}^{\top} & y_{k+1}^{\top} & \ldots & y_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$
$$
\boldsymbol{Y}_r=\left[\begin{array}{llll}
yr_{k}^{\top} & yr_{k+1}^{\top} & \ldots & yr_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\boldsymbol{W}=\left[\begin{array}{llll}
w_{k}^{\top} & w_{k+1}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

$$
\tilde{\boldsymbol{W}}=\left[\begin{array}{llll}
1 & w_{k}^{\top} & \ldots & w_{k+N-1}^{\top}
\end{array}\right]^{\top}
$$

#### Predicted system
$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{A}_{x} x_{k}+\mathcal{B}_{x} \boldsymbol{U} + \mathcal{C}_{x} \boldsymbol{W} \\
\boldsymbol{Y}=\mathcal{A}_{y} x_{k}+\mathcal{B}_{y} \boldsymbol{U} + \mathcal{C}_{y} \boldsymbol{W} + \mathcal{E}_{y} \boldsymbol{W}
\end{array}
$$
with
$$
\mathcal{A}_{x}=\left[\begin{array}{l}
\mathbb{I}_n \\ A^{1}\\  \vdots \\A^{N}
\end{array}\right]
$$

$$
\mathcal{B}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} B & 0 & \ldots & \ldots & 0 \\
A^{1} B & A^{0} B & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} B & A^{N-2} B & \ldots & \ldots & A^{0} B
\end{array}\right]
$$

$$
\mathcal{C}_{x}=\left[\begin{array}{ccccc}
0 & 0 & \ldots & \ldots & 0\\
A^{0} C & 0 & \ldots & \ldots & 0 \\
A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & & \ddots & \ddots & 0 \\
A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

$$
\mathcal{A}_{y}=\left[\begin{array}{l}
D \\ DA^{1} \\ \vdots \\DA^{N-1}
\end{array}\right]
$$

$$
\mathcal{B}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} B & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} B & DA^{0} B & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} B & DA^{N-3} B & \ldots & \ldots & DA^{0} B & 0
\end{array}\right]
$$

$$
\mathcal{C}_{y}=\left[\begin{array}{cccccc}
0 & 0 & \ldots & \ldots & 0 & 0\\
DA^{0} C & 0 & \ldots & \ldots & 0 & 0\\
DA^{1} C & DA^{0} C & \ddots & & \vdots & 0\\
\vdots & \ddots & \ddots & \ddots & \vdots & 0\\
\vdots & & \ddots & \ddots & 0 & 0\\
DA^{N-2} C & DA^{N-3} C & \ldots & \ldots & DA^{0} C & 0
\end{array}\right]
$$

$$
\mathcal{E}_{y} = \left[
  \begin{array}{ccc}
    E & & & & \\
    & \ddots & & & \\
    & & \ddots & & \\
    & & & \ddots & \\
    & & & & E
  \end{array} 
  \right]
$$


#### System with extended $\tilde{\boldsymbol{W}}$

$$
\begin{array}{l}
\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}} \\
\end{array}
$$

$$
\tilde{\mathcal{C}}_{x}=\left[\begin{array}{cccccc}
x_k & 0 & 0 & \ldots & \ldots & 0\\
A x_k & A^{0} C & 0 & \ldots & \ldots & 0 \\
A^2 x_k& A^{1} C & A^{0} C & \ddots & & \vdots \\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & & \ddots & \ddots & 0 \\
A^N x_k & A^{N-1} C & A^{N-2} C & \ldots & \ldots & A^{0} C
\end{array}\right]
$$

#### puried-output-based (POB) affine control laws

$$
y - \hat{y} = \left(
\left[\begin{array}{c|cccc}
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$




$u_{k}=h_{k}+\sum_{\tau=0}^{k} H_{k \tau} v_{\tau}$, where $v_{k} = y_k - \hat{y}_k$

$$
\begin{array}{rll}
u_k &= h_0 + H_{00} (y_k - \hat{y}_k) &= h_0 + H_{00} (Dx_0 +Ew_0)\\
u_{k+1} &= h_1 + H_{10} (y_k - \hat{y}_k) + H_{11} (y_{k+1} - \hat{y}_{k+1}) &= h_1 + H_{10} (Dx_0 +Ew_0) + H_{11} (DAx_0 + DCw_0 + E w_1) \\
\vdots &  & \vdots \\
u_{k+N-1} &= h_{N-1}+\sum_{\tau=0}^{N-1} H_{(N-1) \tau} v_{\tau} &= h_{N-1} +H_{(N-1) 0}(Dx_0 +Ew_0) + \dots + H_{(N-1) (N-1)} (DA^{N-1} x_0 + DA^{N-2}Cw_0 + \dots + DA^{0}Cw_{N-2}) + Ew_{N-1}
\end{array}
$$

$\rightarrow$

$$\boldsymbol{U} = 
\left[\begin{array}{c|cccc}
h_0 & H_{00} & \dots & 0 & 0\\
h_1 & H_{10} & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
h_{N-1} & H_{(N-1)0} & \dots & H_{(N-1)(N-2)} & H_{(N-1)(N-1)}\\
\end{array}\right]
\left(
\left[\begin{array}{c|cccc}
0 & 0 & \dots & 0 & 0\\
DA^0 x_0 & 0 & \dots & 0 & 0\\
DA^1 x_0 & DC & \ddots & 0 & 0\\
\vdots & \vdots & \ddots & \ddots & \vdots\\
DA^{N-1} x_0 & DA^{N-2}C & \dots & DA^{0}C & 0\\
\end{array}\right]
+
\left[\begin{array}{c|cccc}
1 & 0 & \dots & 0 & 0\\
0 & E & 0 & \dots & 0\\
0 & 0 & E & \ddots & 0\\
\vdots &  & \ddots & \ddots & 0\\
0 & 0 & \dots & 0 & E\\
\end{array}\right]
\right)
\left[\begin{array}{c}
1\\
w_0\\
w_1\\
\vdots\\
w_{N-1}\\
\end{array}\right]
$$

$\rightarrow$
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}} $$, where $ \tilde{\mathcal{C}}_y  = \left[\begin{array}{ccccc}
0 & & & & \\
D & 0 & & & \\
& D & 0 & & \\
& & \ddots & \ddots & \\
& & & D & 0
\end{array}\right] \tilde{\mathcal{C}}_x  = \tilde{\mathcal{D}} \tilde{C}_x$


#### Matrix shape
$\boldsymbol{X} \in \mathbb{R}^{(N+1)n \times 1} $,
$\boldsymbol{U} \in \mathbb{R}^{Nm \times 1} $,
$\boldsymbol{Y} \in \mathbb{R}^{Nr \times 1} $,
$\boldsymbol{W} \in \mathbb{R}^{Nd \times 1} $
$\tilde{\boldsymbol{W}} \in \mathbb{R}^{(Nd+1) \times 1} $

$\mathcal{A}_{x} \in \mathbb{R}^{(N+1)n \times n}$,
$\mathcal{B}_{x} \in \mathbb{R}^{(N+1)n \times Nm}$,
$\mathcal{C}_{x} \in \mathbb{R}^{(N+1)n \times Nd}$,
$\mathcal{A}_{y} \in \mathbb{R}^{Nr \times n}$,
$\mathcal{B}_{y} \in \mathbb{R}^{Nr \times Nm}$,
$\mathcal{C}_{y} \in \mathbb{R}^{Nr \times Nd}$,
$\mathcal{E}_{y} \in \mathbb{R}^{Nr \times Nd}$

$\tilde{\mathcal{C}}_{x} \in \mathbb{R}^{(N+1)n \times (Nd+1)}$, $\tilde{\mathcal{C}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{E}}_{y} \in \mathbb{R}^{(Nr + 1) \times (Nd+1)}$, $\tilde{\mathcal{D}} \in \mathbb{R}^{(Nr + 1) \times (N+1)n}$

$\mathcal{H} \in \mathbb{R}^{Nm \times (Nr + 1)}$


### Cost
#### Weighted stage cost
$\beta^{t} (x_{k+t}^{\top} Q x_{k+t} + u_{k+t}^{\top} R u_{k+t} )$
#### Cost
$J_{N}\left(x, \pi_{N}\right):=\sup _{\mathbb{P} \in \mathcal{P}_{\infty}} \mathbb{E}_{\mathbb{P}}\left\{\sum_{t=0}^{N-1} \beta^{t}\left[\boldsymbol{x}_{t}^{\top} Q \boldsymbol{x}_{t}+\boldsymbol{u}_{t}^{\top} R \boldsymbol{u}_{t}\right]+\beta^{N} \boldsymbol{x}_{N}^{\top} Q_{f} \boldsymbol{x}_{N}\right\}$
#### Reformulation with known first and second moments
$$\boldsymbol{U} = \mathcal{H}(\tilde{\mathcal{C}}_y + \tilde{\mathcal{E}}_y) \tilde{\boldsymbol{W}} = H\tilde{\boldsymbol{W}}$$ and $$\boldsymbol{X}=\mathcal{B}_{x} \boldsymbol{U} + \tilde{\mathcal{C}}_{x} \tilde{\boldsymbol{W}}$$

$\tilde{J}_{N}(x, H):=\operatorname{Tr}\left\{\left[(\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]M_w\right\} + \mu_w^{\top} \left[ (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H)^{\top} J_{x} (\tilde{\mathcal{C}}_{x}+\mathcal{B}_{x}H) + H^{\top} J_u H \right]\mu_w$ with $J_x = \left[ \begin{array}{ccccc} \beta^{0}Q&&&&\\ &\beta^{1}Q&&&\\ &&\ddots&& \\ &&&\beta^{N-1}Q&\\ &&&&\beta^{N}Q_f \end{array}\right]$, $J_u = \left[ \begin{array}{cccc} \beta^{0}R&&&\\ &\beta^{1}R&&\\ &&\ddots& \\ &&&\beta^{N-1}R\end{array}\right]$, $M_{w}:=\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)^{\top}\left(1, \mu^{\top}, \ldots, \mu^{\top}\right)+\operatorname{diag}\left(0, \mathbb{I}_{N} \otimes \mathbb{I}_{d}\right)$ and $\mu_w = \left[ \begin{array}{c} 1\\ \mu \\ \vdots \\ \mu \end{array} \right]$

### Chance constraint using wasserstein metric
Constraint on ith element of state: $[x_{k+n}]_i < v_{ubi}$


$\sup _{\mathbb{Q} \in \mathbb{B}_{\varepsilon}\left(\widehat{\mathbb{P}}_{N}\right)} \mathbb{E}^{\mathrm{Q}}[\ell(\xi)]$ with $l(w) = \max_{k \le N-1} \langle a_k, w\rangle + b_k$

equivalent to
$\begin{array}{lll}\inf _{\lambda, s_{i}, \gamma_{i k}} & \lambda \varepsilon+\frac{1}{N} \sum_{i=1}^{N} s_{i} \\ \text { s.t. } & b_{k}+\left\langle a_{k}, \widehat{\xi}_{i}\right\rangle+\left\langle\gamma_{i k}, d-C \widehat{\xi}_{i}\right\rangle \leq s_{i} & \forall i \leq N, \forall k \leq K \\ & \left\|C^{\top} \gamma_{i k}-a_{k}\right\|_{*} \leq \lambda & \forall i \leq N, \forall k \leq K \\ & \gamma_{i k} \geq 0 & \forall i \leq N, \forall k \leq K\end{array}$


#### Define $a_k$ and $b_k$
For example: Constraint on $[x_{k+1}]_1$

We have $x_{k+1} = Ax_0 + Bu_0 + Cw_0$ and $u_0 = h_0 + H_{00}(Dx_0 + Ew_0)$ $\rightarrow$ $[x_{k+1}]_1 = e_1^{\top} (Ax_0 + B(h_0 + H_{00}(Dx_0 + Ew_0)) + Cw_0) = \langle e_1^{\top} (BH_{00}E + C), w_0 \rangle  + e_1^{\top} (Ax_0 + B(h_0 + H_{00}Dx_0))$ 

Similiarly, $x_{k+N} = A^N x_0 + A^{N-1}Bu_0 + \dots + A^{0}Bu_{N-1} + A^{N-1}Cw_0 + \dots + A^{0}Cw_{N-1}$ and $u_{n}=h_{n}+\sum_{\tau=0}^{n} H_{n \tau} v_{\tau}$, where $v_{\tau} = y_{\tau} - \hat{y}_{\tau} = DA^{\tau}x_0 + DA^{\tau-1}Cw_0 + \dots + DA^{0}C w_{\tau-1} + E w_{\tau}$

$\rightarrow$ $[x_{k+N}]_i = e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B \underbrace{(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau} \underbrace{( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa} + Ew_{\tau})}_{v_\tau}))}_{u_{N- 1 - n}} + \sum_{n=0}^{N-1} A^{n}Cw_{N-1-n}) $

$ [x_{k+N}]_i = \langle e_i^{\top}(BH_{(N-1)(N-1)}E + C), w_{N-1} \rangle + e^{\top}_i (A^Nx_0 + \sum_{n=0}^{N-1}A^{n}B(h_{N-1-n} + \sum_{\tau = 0}^{N - 1 - n} ( H_{n\tau}( DA^{\tau}x_0 + \sum_{\kappa = 0}^{\tau-1} DA^{\kappa}Cw_{\tau-1 - \kappa}))) + \sum_{n=1}^{N-1} A^{n}Cw_{N-1-n})$

In [17]:
def stack_system(A, B, C, D, E, x_init, N):
    '''
    Stack system matrix for N prediction horizon
    
    x_next = A x_k + B u_k + C w_k
    y_k = D x_k + E w_k
    
    '''

    n = np.shape(A)[1]    #  Dimension of state
    m = np.shape(B)[1]    #  Dimension of input
    d = np.shape(C)[1]    #  Dimension of disturbance
    r = np.shape(D)[0]    #  Dimension of output
    
#     print(Nx,Nu,Nw,Ny)
    Nx = (N+1) * n
    Nu = N * m
    Ny = N * r
    Nw = N * d
    
    Ax = np.zeros([Nx,n])
    Bx = np.zeros([Nx,Nu])
    Cx = np.zeros([Nx,Nw])
    Ay = np.zeros([Ny,n])
    By = np.zeros([Ny,Nu])
    Cy = np.zeros([Ny,Nw])
    Ey = np.zeros([Ny,Nw])
    Cx_tilde = np.zeros([Nx, Nw + 1])
    Cy_tilde = np.zeros([Ny + 1, Nw + 1])
    Ey_tilde = np.zeros([Ny + 1, Nw + 1])
    D_tilde = np.zeros([Ny + 1, Nx])
#     H

# Ax
    for i in range(N+1):
        Ax[i*n:(i+1)*n,:] = matrix_power(A,i)
# Bx
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            Bx[(i + 1) * n : (i + 2) * n, (i-j) * m: (i-j+1) * m ] = mat_temp
            mat_temp = A @ mat_temp
# Cx
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cx[(i + 1) * n : (i + 2) * n, (i-j) * d: (i-j+1) * d ] = mat_temp
            mat_temp = A @ mat_temp
# Ay
    for i in range(N):
        Ay[i*r:(i+1)*r,:] = D @ matrix_power(A,i) 
# By
    for i in range(N):
        mat_temp = B
        for j in range (i+1):
            By[(i + 1) * r : (i + 2) * r, (i-j) * m: (i-j+1) * m ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Cy
    for i in range(N):
        mat_temp = C
        for j in range (i+1):
            Cy[(i + 1) * r : (i + 2) * r, (i-j) * d: (i-j+1) * d ] = D @ mat_temp
            mat_temp = A @ mat_temp
# Ey
    for i in range(N):
        Ey[i * r : (i+1)*r, i * d : (i+1) * d] = E
# Cx_tilde
    Cx_tilde[ : , 0 : 1 ] = Ax @ x_init
    Cx_tilde[ : , 1 : ] = Cx
# Cy_tilde
    Cy_tilde[ r : , 0 : 1 ] = Ay @ x_init
    Cy_tilde[ r : , 1 : ] = Cy
# Ey_tilde
    Ey_tilde[0, 0] = 1
    Ey_tilde[1:, 1:] = Ey
# D_tilde
    for i in range(N):
        D_tilde[1 + i * r : 1 + (i+1) * r, i * d : (i+1) * d] = D
        
    return n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde 

In [18]:
Ck = np.array([[1e-3, 0],[0, 0]])
D = np.array([[1, 0]])
E = np.array([[0,1e-3]])

N = 10
n, m, d, r, Nx, Nu, Ny, Nw, Ax, Bx, Cx, Ay, By, Cy, Ey, Cx_tilde, Cy_tilde, Ey_tilde, D_tilde  = stack_system(Ak,Bk,Ck,D,E, x_init, N)

In [19]:
Ax

array([[ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.99137931,  0.0862069 ],
       [-0.0862069 ,  0.86206897],
       [ 0.97540131,  0.15978002],
       [-0.15978002,  0.73573127],
       [ 0.95321854,  0.22182772],
       [-0.22182772,  0.62047696],
       [ 0.92587806,  0.2734048 ],
       [-0.2734048 ,  0.51577085],
       [ 0.89432697,  0.31551087],
       [-0.31551087,  0.42106066],
       [ 0.85941804,  0.34908928],
       [-0.34908928,  0.33578412],
       [ 0.82191536,  0.3750268 ],
       [-0.3750268 ,  0.25937516],
       [ 0.78249999,  0.39415374],
       [-0.39415374,  0.19126938],
       [ 0.74177553,  0.4072446 ],
       [-0.4072446 ,  0.13090863],
       [ 0.70027362,  0.4150191 ],
       [-0.4150191 ,  0.07774497]])

In [20]:
Bx

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.04310345, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.03678656, 0.04310345, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.03102385, 0.03678656, 0.0431034

In [21]:
Cx

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.0

In [22]:
Ay

array([[1.        , 0.        ],
       [0.99137931, 0.0862069 ],
       [0.97540131, 0.15978002],
       [0.95321854, 0.22182772],
       [0.92587806, 0.2734048 ],
       [0.89432697, 0.31551087],
       [0.85941804, 0.34908928],
       [0.82191536, 0.3750268 ],
       [0.78249999, 0.39415374],
       [0.74177553, 0.4072446 ]])

In [23]:
By

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01367024, 0.01109139, 0.007989  , 0.00431034, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01577554, 0.01367024, 0.01109139, 0.007989  , 0.00431034,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.01745446, 0.01577554, 0.01367024, 0.01109139, 0.007989  ,
        0.00431034, 0.        , 0.        , 0.        , 0.        ],
       [0.01875134, 0.01745446, 0.0157755

In [24]:
Cy

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.001     , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00099138, 0.        , 0.001     , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0009754 , 0.        , 0.00099138, 0.        , 0.001     ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

In [25]:
Ey

array([[0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.

In [26]:
Cx_tilde

array([[ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.98275862e+00,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e

In [27]:
Cy_tilde

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.98275862e+00, 1.00000000e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.0000

In [28]:
Ey_tilde

array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   

In [29]:
D_tilde

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [30]:
# Decision varibale H
# h vector
H_cal_dec = cp.Variable((Nu , 1))
print(H_cal_dec)
# \mathcal{H} matrix
for i in range(N):
    H_col = cp.Variable((Nu - (i * m) , r))
    if i > 0:
        H_col = cp.vstack([np.zeros([i * m, r]), H_col])
    print(H_col)
    H_cal_dec = cp.hstack([H_cal_dec,H_col])
print(H_cal_dec)

var0
var1
Vstack([[0.]], var3)
Vstack([[0.]
 [0.]], var6)
Vstack([[0.]
 [0.]
 [0.]], var9)
Vstack([[0.]
 [0.]
 [0.]
 [0.]], var12)
Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]], var15)
Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var18)
Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var21)
Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var24)
Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var27)
Hstack(Hstack(Hstack(Hstack(Hstack(Hstack(Hstack(Hstack(Hstack(Hstack(var0, var1), Vstack([[0.]], var3)), Vstack([[0.]
 [0.]], var6)), Vstack([[0.]
 [0.]
 [0.]], var9)), Vstack([[0.]
 [0.]
 [0.]
 [0.]], var12)), Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]], var15)), Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var18)), Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var21)), Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var24)), Vstack([[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], var27))


In [31]:
print(np.shape(H_cal_dec))

(10, 11)


In [32]:
H_cal_dec

Expression(AFFINE, UNKNOWN, (10, 11))

In [33]:
# H_cal_dec = cp.Variable((Nu , 1 + Ny)) # Ny = N * r
# H = H_cal_dec @ (Cy_tilde + Ey_tilde)
H = cp.Variable((Nu , Nw + 1))

In [34]:
# Define loss function
beta = 0.95


Jx = np.zeros([(N+1)* n, (N+1)* n])
for i in range(N):
    Jx[i * n : (i+1) * n, i * n : (i+1) * n] = beta**i * Q
Jx[ N * n: , N * n:] = beta ** N * Qf

Ju = np.zeros([N * m, N * m])
for i in range(N):
    Ju[i * m : (i+1) * m, i * m : (i+1) * m] = beta**i * R

    
mu =np.zeros([d,1])
mu_w = np.vstack([1]+ [mu] * N)
M_w = mu_w @ mu_w.T + np.diag([0] + [1] * N * d)

# loss_func = cp.trace( ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ M_w ) + mu_w.T @ ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ mu_w
# loss_func = cp.trace( ((Cx_tilde+Bx @ H).T @ Jx @ (Cx_tilde+Bx @ H) +  H.T @ Ju @ H ) @ M_w ) 
# loss_func = cp.trace( (Cx_tilde.T @ Jx @ Cx_tilde +  H.T @ (Ju + Bx.T @ Jx @ Bx)@ H + 2 * Cx_tilde.T @ Jx @ Bx @ H) @ M_w ) 
loss_func = cp.trace( (H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w ) 
# loss_func = cp.trace( cp.quad_form(H, Ju + Bx.T @ Jx @ Bx) @ M_w ) 
# loss_func = cp.trace( cp.quad_form(H,Ju + Bx.T @ Jx @ Bx)@ M_w ) 

In [35]:
H_new_matrix = []
for i in range(Nw+1):
    H_new_matrix += [cp.Variable([Nu,1])]
H_new = cp.hstack(H_new_matrix)
print(H_new.shape)

(10, 21)


In [36]:
np.linalg.eig((Ju + Bx.T @ Jx @ Bx))

(array([1.04191564, 0.96750389, 0.91562287, 0.86770817, 0.82269875,
        0.78019971, 0.63144026, 0.73998379, 0.66575349, 0.70188386]),
 array([[-0.88461387, -0.44114778,  0.12946624,  0.06381237, -0.03627689,
          0.02160804,  0.00127516, -0.01281276, -0.00356895, -0.00720868],
        [-0.37088068,  0.86096136,  0.32574815,  0.10541032, -0.05225878,
          0.02917132,  0.00164242, -0.01670885, -0.00455137, -0.00923658],
        [-0.21578327,  0.21284515, -0.9114941 ,  0.25939038, -0.08674329,
          0.04238886,  0.00214777, -0.02279756, -0.00593227, -0.01220281],
        [-0.13848629,  0.10943448, -0.18490172, -0.94033616, -0.2096994 ,
          0.07058131,  0.00286327, -0.03338393, -0.00795729, -0.01683262],
        [-0.09160917,  0.06546115, -0.09030781, -0.16006112,  0.95992712,
          0.16799972,  0.00391869, -0.05573917, -0.01112083, -0.02485292],
        [-0.060284  ,  0.0407801 , -0.05156917, -0.07409753,  0.13584894,
         -0.974125  ,  0.00557466, -0.13090

In [37]:
eigval,eigvec = np.linalg.eig((Ju + Bx.T @ Jx @ Bx))

In [38]:
eigval_mat =  np.diag(eigval)

In [39]:
Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ np.linalg.inv(eigvec)

array([[-1.99840144e-15, -1.07552856e-16, -3.08780779e-16,
        -1.56125113e-16, -1.56125113e-16, -9.54097912e-17,
        -5.55111512e-17, -4.85722573e-17, -2.47198095e-17,
        -2.27682456e-18],
       [-1.97758476e-16, -1.22124533e-15, -1.06858966e-15,
        -2.04697370e-16, -4.52762827e-16, -4.78783679e-16,
        -3.29597460e-16, -2.17707796e-16, -9.49761103e-17,
         5.63785130e-18],
       [-3.74700271e-16, -1.31145095e-15,  4.44089210e-16,
        -1.59594560e-16,  1.82145965e-16,  4.16333634e-16,
        -7.37257477e-17,  8.67361738e-19, -1.25333771e-16,
        -4.94396191e-17],
       [-1.24900090e-16, -3.29597460e-16, -2.56739074e-16,
        -2.22044605e-16, -8.04911693e-16, -5.39499001e-16,
         3.94649591e-16, -1.11889664e-16,  6.72205347e-17,
        -2.02095285e-16],
       [-1.54390389e-16, -5.18682319e-16,  4.68375339e-17,
        -7.37257477e-16,  9.99200722e-16,  2.37657116e-16,
        -1.57859836e-16,  9.71445147e-17,  1.74339709e-16,
         5.

In [40]:
Ju + Bx.T @ Jx @ Bx - eigvec @ eigval_mat @ eigvec.T

array([[-2.88657986e-15,  1.63064007e-16,  6.10622664e-16,
         4.07660017e-16,  6.24500451e-17,  1.16226473e-16,
         1.21430643e-16,  1.41379963e-16,  8.58688121e-17,
         1.29020059e-17],
       [ 2.18575158e-16,  9.99200722e-16, -3.13291060e-15,
        -6.83481050e-16, -7.75421394e-16, -4.89192020e-16,
        -8.76035355e-17, -3.90312782e-16, -1.59594560e-16,
        -1.39428399e-16],
       [ 6.17561557e-16, -3.12944115e-15,  1.11022302e-16,
        -1.79370407e-15, -1.68441650e-15, -4.52762827e-16,
        -7.39859563e-16, -2.42861287e-17,  6.41847686e-17,
         1.77375475e-16],
       [ 3.88578059e-16, -6.97358837e-16, -1.76594850e-15,
        -1.44328993e-15, -5.18682319e-16, -1.35828848e-15,
        -1.74339709e-16,  9.53230550e-16,  3.17454396e-16,
        -7.95804395e-17],
       [ 6.24500451e-17, -7.73686670e-16, -1.68268177e-15,
        -4.90926744e-16,  6.66133815e-16,  1.02175213e-15,
         1.34961486e-15, -2.44596010e-16,  2.19876201e-16,
        -7.

In [41]:
loss_func = 0

N_eig = np.shape(eigval)[0]
I = np.diag([1]*N_eig)
for i in range(N_eig):
    # Tr(H.T @ (Ju + Bx.T @ Jx @ Bx)@ H ) @ M_w
    loss_func += eigval[i] * cp.quad_form(H_new_matrix[i], I) # When M_w is identity matrix. Otherwise reformulate system matrix or this line



In [42]:
np.ones(3)

array([1., 1., 1.])

In [43]:
N_sample = 3

w_sample = []
for i in range(N_sample):
    w_temp = np.random.randn(N*d)
    w_sample += [w_temp]
w_sample_matrix = np.array(w_sample).T

W_sample_matrix_ext = np.vstack( [np.ones([1, N_sample]),w_sample_matrix])

In [44]:
w_sample[0]

array([-0.69415517, -0.40636674, -2.42405734, -2.84312363,  1.63568191,
       -1.23331841, -0.55736659,  0.26714404,  0.1132983 , -0.71823928,
        0.57388392,  0.61452888,  0.65884311,  0.36846456,  0.69347891,
       -0.67956186, -0.11834368,  0.58909701,  1.78054099, -1.31930321])

In [45]:
w_sample_matrix

array([[-0.69415517,  0.8613525 , -0.94276869],
       [-0.40636674, -0.7109658 , -0.32691227],
       [-2.42405734,  0.0304001 ,  0.56157782],
       [-2.84312363,  0.81780373, -0.84793689],
       [ 1.63568191,  0.22288011, -1.54531191],
       [-1.23331841,  0.71430129,  2.40463298],
       [-0.55736659, -0.87357899,  1.81362427],
       [ 0.26714404,  1.21782475,  0.96351917],
       [ 0.1132983 ,  0.43233166,  0.70335391],
       [-0.71823928, -0.55749734, -0.26509373],
       [ 0.57388392, -1.07561025,  0.94192752],
       [ 0.61452888,  1.4559228 , -0.09351583],
       [ 0.65884311,  0.75532056,  1.06117505],
       [ 0.36846456,  0.66395016,  0.59396755],
       [ 0.69347891, -1.1719764 ,  1.05080595],
       [-0.67956186,  0.93655245,  0.26019884],
       [-0.11834368,  2.29653151, -0.42164243],
       [ 0.58909701, -0.13794191, -0.56741804],
       [ 1.78054099,  0.79778701,  0.06798458],
       [-1.31930321, -0.30964381,  0.98787472]])

In [46]:
W_sample_matrix_ext

array([[ 1.        ,  1.        ,  1.        ],
       [-0.69415517,  0.8613525 , -0.94276869],
       [-0.40636674, -0.7109658 , -0.32691227],
       [-2.42405734,  0.0304001 ,  0.56157782],
       [-2.84312363,  0.81780373, -0.84793689],
       [ 1.63568191,  0.22288011, -1.54531191],
       [-1.23331841,  0.71430129,  2.40463298],
       [-0.55736659, -0.87357899,  1.81362427],
       [ 0.26714404,  1.21782475,  0.96351917],
       [ 0.1132983 ,  0.43233166,  0.70335391],
       [-0.71823928, -0.55749734, -0.26509373],
       [ 0.57388392, -1.07561025,  0.94192752],
       [ 0.61452888,  1.4559228 , -0.09351583],
       [ 0.65884311,  0.75532056,  1.06117505],
       [ 0.36846456,  0.66395016,  0.59396755],
       [ 0.69347891, -1.1719764 ,  1.05080595],
       [-0.67956186,  0.93655245,  0.26019884],
       [-0.11834368,  2.29653151, -0.42164243],
       [ 0.58909701, -0.13794191, -0.56741804],
       [ 1.78054099,  0.79778701,  0.06798458],
       [-1.31930321, -0.30964381,  0.987

In [47]:
Cx_tilde

array([[ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.98275862e+00,  1.00000000e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e

In [48]:
# H = H_dec @ (Cy_tilde + Ey_tilde)

In [49]:
X_constraint = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde) @ W_sample_matrix_ext

In [50]:
# select i th element from X_

In [51]:
print(np.shape(X_constraint))

(22, 3)


In [52]:
constraint = []
constraint += [H_new == np.linalg.inv(eigvec) @ H ]
constraint += [H == H_cal_dec @ (Cy_tilde + Ey_tilde) ]

In [53]:
d_supp = np.ones([N*d, 1])
C_supp = np.diag([1]*N*d)

epsilon = 0.1

lambda_var = cp.Variable()

gamma_matrix = []
for i in range(N_sample):
    for j in range(N):
        gamma_var = cp.Variable([N*d,1])
        gamma_matrix += [gamma_var]

si_var = cp.Variable([N_sample,1])
for i in range(N_sample):
    for j in range(N):
        constraint_temp = X_constraint[n * (j+1),i] + gamma_matrix[i * N_sample + j].T @ (d_supp - C_supp @ w_sample_matrix[:, i])
        constraint += [constraint_temp <= si_var[i,0]]
        
        
ak_matrix = (Bx @ H_cal_dec @ (Cy_tilde + Ey_tilde) + Cx_tilde)[:,1:]
for i in range(N_sample):
    for j in range(N):
        constraint_temp = C_supp.T @ gamma_matrix[i * N_sample + j] - ak_matrix[n * (j+1):n * (j+1)+1,:].T
        constraint += [cp.norm_inf(constraint_temp) <= lambda_var]
        
for i in range(N_sample):
    for j in range(N):
        constraint += [gamma_matrix[i * N_sample + j] >= 0]

        
constraint += [lambda_var * epsilon + 1/N_sample * cp.sum(si_var) <= 0.5]
# supp_constraint_matrix = 

In [54]:
obj = cp.Minimize(loss_func)
prob = cp.Problem(obj,constraint)
prob.solve()

110418.89556313632

In [55]:
H_cal_dec.value

array([[-1.87825749e+05,  9.37587993e+04,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 9.47669448e+04, -4.73832570e+04, -2.17234393e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 6.82586986e+04, -3.41294808e+04,  3.24192444e-02,
         1.01895921e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 3.68282413e+04, -1.84140501e+04, -1.00120641e-01,
         4.76146200e-02, -1.85797849e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.24118800e-01, -1.13483137e-02, -8.01896142e-02,
         5.43617298e-02, -3.13253332e-02,  6.09254655e-03,
  

In [56]:
H.value @ W_sample_matrix_ext

array([[-3.46251392e+02, -3.74810233e+02, -3.38801836e+02],
       [ 1.92557983e+01,  3.36875604e+01,  1.54906072e+01],
       [ 1.38685750e+01,  2.42651574e+01,  1.11575415e+01],
       [ 7.48306878e+00,  1.30916670e+01,  6.02011461e+00],
       [ 1.09249901e-04, -7.75985002e-05,  3.21962082e-04],
       [ 1.27217531e-04, -1.02554308e-05,  2.99993386e-04],
       [ 2.67262024e-01, -1.57581001e+00,  1.39682217e+00],
       [-1.02283974e+00,  1.64559179e+00,  4.17588038e+00],
       [ 6.24667195e-01, -4.46877179e-01,  1.60109091e+00],
       [-2.66278471e+00,  3.08659830e+00,  1.06867031e+00]])